In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
# from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix
from sklearn.linear_model import LogisticRegression

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder

In [3]:
X_train = pd.read_csv("../outputs/imputed_X_train_SMOTE_0_35.csv")
y_train = pd.read_csv("../outputs/imputed_y_train_SMOTE_0_35.csv")
X_test = pd.read_csv("../outputs/imputed_X_test_scaled_0_35.csv")
y_test = pd.read_csv("../outputs/imputed_y_test.csv")

# def map_df(data):

#     data["GenderCategory"] = data["GenderCategory"].map({1: "Female", 2: "Male"})
#     data["RaceCategory"] = data["RaceCategory"].map({1:"White", 2:"Black", 3:"Unknown"})
#     data["EthnicityCategory"] = data["EthnicityCategory"].map({1: "Non-Hispanic", 2:"Hispanic", 3: "Unknown"})
#     data["Region"] = data["Region"].map({1: "Northeast", 2: "Midwest", 3: "South", 4: "West", 5: "Unknown"})

#     return data

# X_train = map_df(X_train.copy(deep = True))
# X_test = map_df(X_test.copy(deep = True))

In [4]:
X_train.head()

,GenderCategory,RaceCategory,EthnicityCategory,Age,Region,Glipizide_Final,Glimepiride_Final,Glyburide_Final,Metformin_Final,Pioglitazone_Final,...,Eye_Disorder_Final,Atherosclerosis_Final,Alzheimer_Final,FootUlcer_Final,Abnormal_Glucose_Final,DMScreen_Final,A1C_Final,GlucoseTest_Final,DM_Drugs,AnyLab
0,Female,White,Non-Hispanic,-0.387508,West,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Male,White,Non-Hispanic,-1.026402,South,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,Male,White,Non-Hispanic,0.331247,Midwest,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Female,White,Non-Hispanic,1.129864,South,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
4,Male,White,Non-Hispanic,0.890279,Northeast,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [5]:
X_test.head()

,GenderCategory,RaceCategory,EthnicityCategory,Age,Region,Glipizide_Final,Glimepiride_Final,Glyburide_Final,Metformin_Final,Pioglitazone_Final,...,Eye_Disorder_Final,Atherosclerosis_Final,Alzheimer_Final,FootUlcer_Final,Abnormal_Glucose_Final,DMScreen_Final,A1C_Final,GlucoseTest_Final,DM_Drugs,AnyLab
0,Female,White,Non-Hispanic,-0.227785,Northeast,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Female,White,Non-Hispanic,0.171524,Midwest,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Female,White,Non-Hispanic,2.008343,South,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Male,White,Non-Hispanic,-0.307646,South,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Male,White,Non-Hispanic,0.490971,Midwest,0,0,1,1,0,...,0,0,0,0,0,0,0,0,1,0


In [6]:
numerical_col = ["Age"]
categorical_col = list(X_train.columns)
categorical_col.remove("Age")
X_train[categorical_col] = X_train[categorical_col].astype('category')

In [7]:
categorical_features = X_train.columns[X_train.dtypes == 'category']
cat_col_numbers = [X_train.columns.get_loc(col) for col in X_train.select_dtypes(include="category")]
categorical_features

Index(['GenderCategory', 'RaceCategory', 'EthnicityCategory', 'Region',
       'Glipizide_Final', 'Glimepiride_Final', 'Glyburide_Final',
       'Metformin_Final', 'Pioglitazone_Final', 'Rosiglitazone_Final',
       'Beta_Blockers_Final', 'ACE_Inhibitors_Final', 'ARB_Final',
       'Diuretics_Final', 'PPI_Final', 'Levothyroxine_Final', 'CCB_Final',
       'Vasodilators_Final', 'Statins_Final', 'Anti_Platelets_Final',
       'Anti_Coagulants_Final', 'Steroids_Final', 'Heart_Disease_Final',
       'Hypothyroid_Final', 'Anemia_Final', 'Kidney_Disease_Final',
       'GERD_Final', 'Neuropathy_Final', 'Eye_Disorder_Final',
       'Atherosclerosis_Final', 'Alzheimer_Final', 'FootUlcer_Final',
       'Abnormal_Glucose_Final', 'DMScreen_Final', 'A1C_Final',
       'GlucoseTest_Final', 'DM_Drugs', 'AnyLab'],
      dtype='object')

In [8]:
lbl = LabelEncoder()
for col in categorical_col:
    X_train[col] = lbl.fit_transform(X_train[col])
    X_test[col] = lbl.transform(X_test[col])

## Feature selection

In [9]:
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import chi2

# def select_features(X_train, y_train, X_test):
#     fs = SelectKBest(score_func=chi2, k=10)
#     fs.fit(X_train.loc[:,categorical_col], y_train)
#     X_train_fs = fs.transform(X_train.loc[:,categorical_col])
#     X_test_fs = fs.transform(X_test.loc[:,categorical_col])

#     X_train_fs = pd.DataFrame(X_train_fs, columns = fs.f)




#     return X_train_fs, X_test_fs, fs


# a,b,c = select_features(X_train,y_train,X_test)
# for i in range(len(c.scores_)):
#  print('Feature %d: %f' % (i, c.scores_[i]))
# # plot the scores
# sns.barplot(x = [i for i in range(len(c.scores_))], y = c.scores_)

# Modelling

In [10]:
clf = LogisticRegression(penalty="l1", solver = "saga", max_iter=5000)
clf.fit(X_train,y_train.values.ravel())
clf_coeff = pd.DataFrame(clf.coef_, columns=X_train.columns)
clf_coeff = clf_coeff.T.reset_index()
clf_coeff[clf_coeff[0] == 0].index

Index([9, 18, 19, 30, 32, 34, 35, 37], dtype='int64')

In [11]:
# X_train = X_train.drop(X_train.iloc[ :, [5, 18, 24, 25, 28, 32, 34, 36]].columns, axis = 1).reset_index(drop = True)
# X_test = X_test.drop(X_test.iloc[:, [5, 18, 24, 25, 28, 32, 34, 36]].columns, axis = 1).reset_index(drop=True)

# cat_col_numbers = [X_train.columns.get_loc(col) for col in X_train.select_dtypes(include="category")]

In [12]:
n_splits = 5

model_params = {
    "cat_1" : {
        "objective" : "CrossEntropy",
        "eval_metric" : 'AUC',
        "learning_rate" : 0.001, 
        "max_depth" : 16,
        # "random_state" : 56,
        "subsample" : 0.35,
        # 'early_stopping_rounds': 500,
        # 'ignored_features' : [4, 5, 18, 24, 25, 28, 32, 34, 36],
        "n_estimators": 10,
        "cat_features" : cat_col_numbers,
        "verbose" : 0        
    },


    "hgb_1" : {
        "learning_rate":0.015,
        "n_iter_no_change":100,
        "l2_regularization" : 0.02,
        "random_state" : 42,
        "categorical_features" : cat_col_numbers,
        
    },

    "lgbm_1" :
    {
        "objective" : "binary",
        "metric" : 'AUC',
        "learning_rate" : 0.018, 
        "max_depth" : 16,
        "random_state" : 56,
        "reg_alpha" : 0.01246,
        "reg_lambda": 0.023483,
        "subsample" : 0.35,
        "colsample_bytree" : 0.8,
        "verbose" : 0,
        "num_leaves" : 11,
        "categorical_features" : cat_col_numbers,
        "verbose" : -1,
        "class_weight" : "balanced"

    }
}

hgb_1 = HistGradientBoostingClassifier(**model_params["hgb_1"])
lgbm_1 = LGBMClassifier(**model_params["lgbm_1"])
# cat_1 = CatBoostClassifier(**model_params["cat_1"])

In [13]:
oofs_dict = {model + "_oof" : np.zeros(len(X_train)) for model in model_params.keys()}

In [14]:
def eval(y_true, y_pred, y_pred_proba):
    acc_score = accuracy_score(y_true, y_pred)
    conf_mat = confusion_matrix(y_true,y_pred)

    prec_score = precision_score(y_true, y_pred) 
    rec_score = recall_score(y_true, y_pred)
    F1_score = f1_score(y_true, y_pred)

    roc_auc = roc_auc_score(y_true, y_pred)
    roc_auc_predict_proba = roc_auc_score(y_true, y_pred_proba)

    print(f'Accuracy: {acc_score}')

    print("Precision_score :", prec_score)
    print("recall_score :", rec_score)
    print("f1_score:", F1_score)

    print(f'ROC AUC score: {roc_auc}')
    print(f'ROC AUC PROBA: {roc_auc_predict_proba}')

    print(conf_mat)

    print("==" * 25)

In [15]:
kf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
model_auc = {model + "_auc" : []  for model in model_params.keys()}


for num_fold, (train_index, val_index) in enumerate(kf.split(X_train,y_train.values)):
    
    print("="*20, "Fold : ", num_fold, "="*20)
    X_train_fold = X_train.iloc[train_index]
    X_val_fold = X_train.iloc[val_index]
    y_train_fold = y_train.iloc[train_index].values.ravel()
    y_val_fold = y_train.iloc[val_index].values.ravel()
    

    # cat_1.fit(X_train_fold,y_train_fold, eval_set=(X_val_fold,y_val_fold))
    # oofs_dict["cat_1_oof"][val_index] = cat_1.predict(X_val_fold)
    # y_pred = cat_1.predict(X_val_fold)
    # y_pred_proba = cat_1.predict_proba(X_val_fold)[:, 1]
    # # print('Fold', num_fold, 'CAT_1: ')
    # model_auc["cat_1_auc"].append(roc_auc_score(y_val_fold, y_pred))
    # eval(y_val_fold,y_pred, y_pred_proba)


    hgb_1.fit(X_train_fold, y_train_fold)
    oofs_dict["hgb_1_oof"][val_index] = hgb_1.predict(X_val_fold)
    y_pred = hgb_1.predict(X_val_fold)
    y_pred_proba = hgb_1.predict_proba(X_val_fold)[:, 1]
    model_auc["hgb_1_auc"].append(roc_auc_score(y_val_fold, y_pred))
    # print('Fold', num_fold, 'HGB_1: ')
    eval(y_val_fold,y_pred, y_pred_proba)
    
    
    lgbm_1.fit(X_train_fold, y_train_fold)
    oofs_dict["lgbm_1_oof"][val_index] = lgbm_1.predict(X_val_fold)
    y_pred = lgbm_1.predict(X_val_fold)
    y_pred_proba = lgbm_1.predict_proba(X_val_fold)[:, 1]
    # print('Fold', num_fold, 'LGBM_1: ')
    model_auc["lgbm_1_auc"].append(roc_auc_score(y_val_fold, y_pred))
    eval(y_val_fold,y_pred, y_pred_proba)


==================== Fold :  0 ====================
Accuracy: 0.8571428571428571
Precision_score : 0.860655737704918
recall_score : 0.963302752293578
f1_score: 0.9090909090909091
ROC AUC score: 0.7579671656204732
ROC AUC PROBA: 0.8660067600193143
[[ 21  17]
 [  4 105]]
Accuracy: 0.782312925170068
Precision_score : 0.9052631578947369
recall_score : 0.7889908256880734
f1_score: 0.8431372549019608
ROC AUC score: 0.7760743602124578
ROC AUC PROBA: 0.8631096088845968
[[29  9]
 [23 86]]
==================== Fold :  1 ====================
Accuracy: 0.8299319727891157
Precision_score : 0.8387096774193549
recall_score : 0.9541284403669725
f1_score: 0.8927038626609441
ROC AUC score: 0.7139063254466442
ROC AUC PROBA: 0.8105987445678416
[[ 18  20]
 [  5 104]]
Accuracy: 0.8095238095238095
Precision_score : 0.8648648648648649
recall_score : 0.8807339449541285
f1_score: 0.8727272727272727
ROC AUC score: 0.7429985514244326
ROC AUC PROBA: 0.8067358763882182
[[23 15]
 [13 96]]
==================== Fold :

In [16]:
# sum(model_auc["cat_1_auc"])/5, sum(model_auc["hgb_1_auc"])/5, sum(model_auc["lgbm_1_auc"])/5


sum(model_auc["hgb_1_auc"])/5, sum(model_auc["lgbm_1_auc"])/5

(0.7298862599924888, 0.7792026575102384)

# HGB is better classifier than LGBM

In [17]:
# import optuna

# def objective(trial,data=X_train,target=y_train.values.ravel()):
#     train_x, val_x, train_y, val_y = train_test_split(data, target, test_size=0.33,random_state=42, stratify=target)
#     params = {
#         'l2_regularization': trial.suggest_float('l2_regularization',0.1,10, log = True),
#         'early_stopping': trial.suggest_categorical('early_stopping', [False, True]),
#         'learning_rate': trial.suggest_float('learning_rate', 0.0001,1, log = True),
#         # 'max_iter': trial.suggest_categorical('max_iter', [750, 1000, 1250, 1500]),
#         'max_depth': trial.suggest_int('max_depth', 2,6),
#         'max_bins': trial.suggest_int('max_bins', 5,9),
#         'min_samples_leaf': trial.suggest_int('min_samples_leaf', 3,11),
#         'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 5,11),
#         "categorical_features" : cat_col_numbers,
#         "class_weight" : "balanced"
#     }

#     model = HistGradientBoostingClassifier(**params)
#     model.fit(train_x, train_y)
#     prediction_proba = model.predict_proba(val_x)[:,1]
#     auc = roc_auc_score(val_y, prediction_proba)
#     print(auc)
#     return auc

In [18]:
# %%time
# study = optuna.create_study(direction='maximize')
# optuna.logging.set_verbosity(optuna.logging.ERROR)
# study.optimize(objective, n_trials=100)
# print('Number of finished trials:', len(study.trials))
# print('Best trial:', study.best_trial.params)

In [19]:
# hgb_1 = HistGradientBoostingClassifier(**study.best_trial.params)
# hgb_1.fit(X_train,y_train.values.ravel())

# y_pred = hgb_1.predict(X_train)
# y_pred_proba = hgb_1.predict_proba(X_train)[:,1]
# eval(y_train, y_pred, y_pred_proba)


# y_pred = hgb_1.predict(X_test)
# y_pred_proba = hgb_1.predict_proba(X_test)[:,1]
# eval(y_test, y_pred, y_pred_proba)

In [20]:
# import joblib
# filename = '../models/HGB_ROC AUC PROBA: 0.8551.sav'
# joblib.dump(hgb_1, filename)
 

# LGBM

In [39]:
def objective(trial,data=X_train,target=y_train.values.ravel()):
    train_x, val_x, train_y, val_y = train_test_split(data, target, test_size=0.33,random_state=42, stratify=target)
    params = {
        'reg_alpha': trial.suggest_float('reg_alpha',1e-2,1, log = True),
        'reg_lambda': trial.suggest_float('reg_lambda',1e-2,1, log = True),
        'learning_rate': trial.suggest_float('learning_rate', 0.001,1, log = True),
        'subsample': trial.suggest_float('subsample', 0.01,1, log = True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01,1, log = True),
        # 'max_iter': trial.suggest_categorical('max_iter', [750, 1000, 1250, 1500]),
        'max_depth': trial.suggest_int('max_depth', 2,11),
        # "n_estimator" : trial.suggest_int("n_estimators", 50,70),
        'max_bins': trial.suggest_int('max_bins', 5,9),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 3,9),
        'num_leaves': trial.suggest_int('num_leaves', 5,11),
        # "categorical_features" : trial.suggest_categorical("categorical_features", [cat_col_numbers]),
        "class_weight" : trial.suggest_categorical("class_weight", ["balanced"]),
        "objective" : trial.suggest_categorical("objective", ["binary"]),
        "metric" : trial.suggest_categorical("metric", ["AUC"]),
        "verbose" : trial.suggest_categorical("verbose", [-1])
    }

    model = LGBMClassifier(**params)
    model.fit(train_x, train_y)
    prediction_proba = model.predict_proba(val_x)[:, 1]
    auc = roc_auc_score(val_y, prediction_proba)
    print(auc)
    return auc

In [40]:
%%time
import optuna

study = optuna.create_study(direction='maximize')
optuna.logging.set_verbosity(optuna.logging.ERROR)
study.optimize(objective, n_trials=100)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

0.8531967721911856
0.8635275339185954
0.8573202092755166
0.8709763234902901
0.8386538973131151
0.8445508557240402
0.8583399840383081
0.8365256717212025
0.8432207147290947
0.8737696195796755
0.8483639265762171
0.8258402057284738
0.8267269663917709
0.8266382903254411
0.8531967721911855
0.8258845437616388
0.827259022789749
0.8411368271703467
0.8335550234991576
0.8681830274009046
0.8719517602199167
0.8719517602199167
0.8261949099937927
0.8251751352310012
0.8720404362862464
0.8224705152079453
0.8482309124767224
0.8256628535958145
0.8505808282344595
0.8249091070320121
0.8575418994413407
0.8719517602199166
0.8504921521681299
0.8257515296621442
0.8218497827436375
0.8753214507404452
0.8582069699388134
0.879178859625787
0.8295646005143213
0.8582069699388134
0.8344417841624545
0.8749667464751264
0.8770062960007095
0.8582956460051433
0.8270816706570897
0.8766515917353906
0.8270816706570897
0.8813070852176997
0.8321362064378824
0.8821938458809966
0.8814844373503591
0.8643699565487276
0.870931985457

In [41]:
lgbm_1 = LGBMClassifier(**study.best_trial.params)
lgbm_1.fit(X_train,y_train.values.ravel())

y_pred = lgbm_1.predict(X_train)
y_pred_proba = lgbm_1.predict_proba(X_train)[:,1]
eval(y_train, y_pred, y_pred_proba)


y_pred = lgbm_1.predict(X_test)
y_pred_proba = lgbm_1.predict_proba(X_test)[:,1]
eval(y_test, y_pred, y_pred_proba)

Accuracy: 0.819918144611187
Precision_score : 0.8841121495327103
recall_score : 0.8710865561694291
f1_score: 0.8775510204081634
ROC AUC score: 0.7723853833478724
ROC AUC PROBA: 0.8781913346903168
[[128  62]
 [ 70 473]]
Accuracy: 0.8272727272727273
Precision_score : 0.9306122448979591
recall_score : 0.8507462686567164
f1_score: 0.8888888888888888
ROC AUC score: 0.7882763601348098
ROC AUC PROBA: 0.8423206547905633
[[ 45  17]
 [ 40 228]]


In [ ]:
# import joblib
# filename = '../models/LGBM/LGBM_ROC AUC PROBA: 0.854.sav'
# joblib.dump(lgbm_1, filename)
 

['../models/LGBM/LGBM_ROC AUC PROBA: 0.854.sav']

In [ ]:
# lgbm_1.booster_.save_model("../models/LGBM_ROC AUC PROBA: 0.8601.txt")

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
estimators = [("h", hgb_1), ("l", lgbm_1)]
clf = StackingClassifier(estimators=estimators, final_estimator=LGBMClassifier(**study.best_trial.params))


clf.fit(X_train,y_train.values.ravel())

y_pred = clf.predict(X_train)
y_pred_proba = clf.predict_proba(X_train)[:,1]
eval(y_train, y_pred, y_pred_proba)


y_pred = clf.predict(X_test)
y_pred_proba = clf.predict_proba(X_test)[:,1]
eval(y_test, y_pred, y_pred_proba)


Accuracy: 0.8635743519781719
Precision_score : 0.9317738791423001
recall_score : 0.8802946593001841
f1_score: 0.9053030303030302
ROC AUC score: 0.8480420664921973
ROC AUC PROBA: 0.90557817194921
[[155  35]
 [ 65 478]]
Accuracy: 0.8121212121212121
Precision_score : 0.9364406779661016
recall_score : 0.8246268656716418
f1_score: 0.876984126984127
ROC AUC score: 0.791345690900337
ROC AUC PROBA: 0.8387096774193548
[[ 47  15]
 [ 47 221]]
